# Risk Analysis Assignment 1

* There exist five assest in the portfolio, which is a `payer swap`, and four stocks `(AAPL, MSFT, F and BAC)`
* Investors have 1 million dollars for each stock, and the information of the payer swap is listed below:
|           | Payer Swap  |
|-----------|-------------|
| Notional  | 100 million |
| Maturity  | 10 years    |
| Strike    | 4.2%   |
| Payment Frequency    | Annual   |

* Through the historical data over a period from 31/10/2022 to 30/10/2024, the goal of the project is calculate the 1-day 95% VaR of the portfolio using three different models.

### Import packages

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
np.set_printoptions(precision = 3)

plt.style.use("ggplot")

mpl.rcParams["axes.grid"] = True
mpl.rcParams["grid.color"] = "grey"
mpl.rcParams["grid.alpha"] = 0.25

mpl.rcParams["axes.facecolor"] = "white"

mpl.rcParams["legend.fontsize"] = 8

 # <a id = "top">Contents</a>

## [1. Data Analysis](#p1)
## [2. Parametric VaR Model](#p2)
## [3. Monte Carlo VaR Model](#p3)
## [4. Historical VaR Model](#p4)
## [5. Conclusion](#p5)

##  <a id = "p1"> 1.<font color = "blue">Data Analysis [(back to contents)](#top)

In [3]:
RF_sofr = pd.read_excel("hist_data.xlsm", sheet_name = "SofrCurve")
RF_AAPL = pd.read_excel("hist_data.xlsm", sheet_name = "AAPL")
RF_MSFT = pd.read_excel("hist_data.xlsm", sheet_name = "MSFT")
RF_F = pd.read_excel("hist_data.xlsm", sheet_name = "F")
RF_BAC = pd.read_excel("hist_data.xlsm", sheet_name = "BAC")

> Stock data\
Using relative change


In [4]:
Stock_Change = pd.DataFrame()
Stock_Change["AAPL"] = RF_AAPL["Adj Close"].pct_change()
Stock_Change["MSFT"] = RF_MSFT["Adj Close"].pct_change()
Stock_Change["F"] = RF_F["Adj Close"].pct_change()
Stock_Change["BAC"] = RF_BAC["Adj Close"].pct_change()
Stock_Change.index = RF_AAPL["Date"]
Stock_Change = Stock_Change.dropna()

In [5]:
Stock_Change.head()

,AAPL,MSFT,F,BAC
Date,,,,
2022-11-01,-0.017543,-0.017059,0.002244,0.004439
2022-11-02,-0.037305,-0.035368,-0.025373,-0.003039
2022-11-03,-0.042405,-0.026579,0.015314,-0.005542
2022-11-04,-0.001947,0.033326,0.018854,0.025077
2022-11-07,0.003902,0.029270,0.014064,0.005980


> SOFR curve\
Using absolute change

In [6]:
RF_sofr_new = RF_sofr.drop(["T","Tenor"],axis = 1).T
RF_sofr_new.columns = RF_sofr["Tenor"]
RF_sofr_new.head()

Tenor,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-10-31 00:00:00,0.039191,0.038721,0.038670,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.032350,0.030552,0.028708
2022-11-01 00:00:00,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.046970,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.036010,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02 00:00:00,0.039948,0.039286,0.039100,0.040852,0.044884,0.046580,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03 00:00:00,0.040389,0.039585,0.039350,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04 00:00:00,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.047500,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.037380,0.037152,0.036870,0.036534,0.034424,0.032558,0.030723,0.028933


In [7]:
SOFR_change = pd.DataFrame()
for i in RF_sofr_new.columns:
    change = RF_sofr_new[i].diff()
    SOFR_change = pd.concat([SOFR_change, change],axis = 1)
SOFR_change = SOFR_change.dropna()
SOFR_change.head()

,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-11-01 00:00:00,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,0.000521,0.000439,0.000342,0.000296,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.000230
2022-11-02 00:00:00,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,0.000234,0.000474,0.000404,0.000254,...,0.000053,0.000019,-0.000013,-0.000040,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.000130
2022-11-03 00:00:00,0.000440,0.000299,0.000250,0.000302,0.000397,0.000527,0.000690,0.001098,0.001085,0.000957,...,0.000366,0.000368,0.000388,0.000409,0.000420,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04 00:00:00,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,-0.000394,-0.000497,-0.000448,-0.000322,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
2022-11-07 00:00:00,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,0.000429,0.000536,0.000499,0.000413,...,0.000431,0.000463,0.000487,0.000504,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562


Because the 10-year payer swap pay annually, focusing on 1Y to 10Y SOFR rate is enough. Furthermore, merge them into one dataframe.

In [15]:
Risk_Factor_Change = pd.concat([Stock_Change,SOFR_change.loc[:,"1Y":"10Y"]],axis = 1).sort_index()
Risk_Factor_Change = Risk_Factor_Change.dropna(subset=['AAPL'])
Risk_Factor_Change = Risk_Factor_Change.interpolate(method = "linear").reset_index(drop = True)

In [16]:
Risk_Factor_Change

,AAPL,MSFT,F,BAC,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y
0,-0.017543,-0.017059,0.002244,0.004439,0.000521,0.000439,0.000342,0.000296,0.000225,0.000124,0.000029,-0.000043,-9.724439e-05,-0.000144
1,-0.037305,-0.035368,-0.025373,-0.003039,0.000234,0.000474,0.000404,0.000254,0.000153,0.000118,0.000109,0.000099,8.994989e-05,0.000091
2,-0.042405,-0.026579,0.015314,-0.005542,0.000690,0.001098,0.001085,0.000957,0.000832,0.000761,0.000704,0.000624,5.446595e-04,0.000495
3,-0.001947,0.033326,0.018854,0.025077,-0.000394,-0.000497,-0.000448,-0.000322,-0.000204,-0.000116,-0.000038,0.000045,1.258218e-04,0.000192
4,0.003902,0.029270,0.014064,0.005980,0.000429,0.000536,0.000499,0.000413,0.000332,0.000313,0.000328,0.000342,3.452884e-04,0.000339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.002543,0.003674,-0.007840,-0.003911,0.000114,0.000371,0.000417,0.000343,0.000257,0.000165,0.000083,0.000032,2.757643e-07,-0.000027
246,-0.013492,0.030678,0.013169,0.003141,0.000150,0.000393,0.000596,0.000743,0.000894,0.000981,0.001022,0.001051,1.077486e-03,0.001100
247,-0.024606,-0.037514,-0.016464,0.022309,-0.000410,-0.000747,-0.000950,-0.001055,-0.001114,-0.001124,-0.001102,-0.001073,-1.046232e-03,-0.001028
248,0.007969,0.005856,-0.122467,-0.036371,-0.000129,-0.000286,-0.000361,-0.000339,-0.000324,-0.000257,-0.000166,-0.000093,-3.667796e-05,0.000011


In [14]:
Risk_Factor_Change.to_excel("change.xlsx")

##  <a id = "p2"> 2.<font color = "blue">Monte Carlo VaR Model [(back to contents)](#top)

Calculate 1-day 95% VaR, which means h=1.

In [18]:
def calculate_PV(df_10y):
    pv01_ls = []
    T = 1
    pv_fix = 0
    for i in range(len(df_10y)):
    
        zero_rate = df_10y.iloc[i]
        DF = np.exp(-zero_rate*T)
        pv_fix = pv_fix + 0.042*DF
        T = T+1
    
    pv_flt = 1-DF
    pv = np.abs(100000000*pv_flt-100000000*pv_fix)
    return pv

def calculate_pv01(df_10y):
    pv01_ls = []
    pv_0 = calculate_PV(df_10y)
    for j in range(len(df_10y)):
        df_10y.iloc[j] = df_10y.iloc[j] + 0.0001
        pv_1 = calculate_PV(df_10y)
        pv01_ls.append(pv_1-pv_0)
        df_10y.iloc[j] = df_10y.iloc[j] - 0.0001
    return pv01_ls

Calculate the weights of 10 risk factors of SOFR curve:

In [19]:
RF_sofr_new1 = RF_sofr_new.reset_index(drop = True)
pv01 = calculate_pv01(RF_sofr_new1.loc[250,"1Y":"10Y"])
pv01

[398.60067670047283,
 763.1796779595315,
 1099.3001080676913,
 1406.65535665676,
 1685.4750986732543,
 1937.1056086532772,
 2163.2335943840444,
 2365.3227567374706,
 2544.7769064716995,
 67059.5262562409]

In [22]:
[mu_p,sigma2_p]

[13354.513854651364, 339423748218.0995]

In [ ]:
#a: weight
mu = np.mean(Risk_Factor_Change, axis =0)
mu_p = sum(mu*np.array(a))
vriance_p = np.dot(np.dot(np.array(a), cov_mat),np.array(a).T)

In [21]:
mu = np.mean(Risk_Factor_Change, axis =0)
a = [1000000]*4 + list(np.array(pv01)*1e4)
mu_p = sum(mu*np.array(a))
sigma2_p = np.dot(np.dot(np.array(a), np.cov(Risk_Factor_Change.T)),np.array(a).T)

In [23]:
VaR = np.abs(mu_p + np.sqrt(sigma2_p)*norm.ppf(0.05,0,1))
VaR

944938.6042235331

##  <a id = "p3"> 3.<font color = "blue">Parametric VaR Model [(back to contents)](#top)

### Method 1: Full revaluation

In [34]:
def calculate_portfolio_value(value_swap, price_stock):
    return value_swap+sum(price_stock)

Simulate 10000 times.

In [35]:
samples = np.random.multivariate_normal(mu, np.cov(Risk_Factor_Change.T), int(1e4))
L_list = []
PnL_swap_df = []
for m in range(int(1e4)):
    #calculate swap value
    sample_swap_change = samples[m][4:14]
    new_rete = np.array(RF_sofr_new1.loc[250, "1Y":"10Y"])+sample_swap_change
    PnL_swap = calculate_PV(pd.DataFrame(new_rete))
    #calculate stock value
    new_price = [1e6 * (1+x) for x in list(samples[m][0:4])]
    #calculate porfolio value
    portfolio_base = calculate_portfolio_value(calculate_PV(RF_sofr_new1.loc[250, "1Y":"10Y"]), [1e6]*4)
    portfolio_new = calculate_portfolio_value(PnL_swap, new_price)
    L_list.append((portfolio_new-portfolio_base)[0])

In [36]:
L_list_MCFull = np.percentile(sorted(L_list),5)
L_list_MCFull

-938441.884206314

### Method 2: Risk-Based

In [37]:
L_ls = []
for m in range(int(1e4)):
    L = np.dot(np.array(a).T,list(samples[m]))
    L_ls.append(L)

In [38]:
L_list_MCRisk = np.percentile(sorted(L_ls),5)
L_list_MCRisk

-932573.5088246102

##  <a id = "p4"> 4.<font color = "blue">Historical VaR Model [(back to contents)](#top)

### Method 1: Full Revaluation

In [39]:
L_ls1 = []
for n in range(len(Risk_Factor_Change)):
    new_rete = np.array(RF_sofr_new1.loc[250, "1Y":"10Y"])+np.array(Risk_Factor_Change.loc[n, "1Y":"10Y"])
    PnL_swap = calculate_PV(pd.DataFrame(new_rete))
    new_price = [1e6 * (1+x) for x in list(Risk_Factor_Change.loc[n, "AAPL":"BAC"])]
    #calculate porfolio value
    portfolio_base = calculate_portfolio_value(calculate_PV(RF_sofr_new1.loc[250, "1Y":"10Y"]), [1e6]*4)
    portfolio_new = calculate_portfolio_value(PnL_swap, new_price)
    L_ls1.append((portfolio_new-portfolio_base)[0])

In [40]:
L_list_HisFull = np.percentile(sorted(L_ls1),5)
L_list_HisFull

-990511.3462687891

### Method 2: Risk-Based

In [41]:
L_ls2 = []
for n in range(len(Risk_Factor_Change)):
    L = np.dot(np.array(a),Risk_Factor_Change.loc[n, :])
    L_ls2.append(L)

In [42]:
L_list_HisRisk = np.percentile(sorted(L_ls2),5)
L_list_HisRisk

-984271.2321714654

##  <a id = "p5"> 5.<font color = "blue">Conclusion [(back to contents)](#top)

In [43]:
result = pd.DataFrame({"Calculate Method": ["Parametric Model", "Monte Carlo-Full Revaluation", "Monte Carlo-Risk Based",
                                            "Historical-Full Revaluation", "Historical-Risk Based"],
                       "VaR Value": [VaR, abs(L_list_MCFull), abs(L_list_MCRisk), abs(L_list_HisFull), abs(L_list_HisRisk)]})
result

,Calculate Method,VaR Value
0,Parametric Model,944938.604224
1,Monte Carlo-Full Revaluation,938441.884206
2,Monte Carlo-Risk Based,932573.508825
3,Historical-Full Revaluation,990511.346269
4,Historical-Risk Based,984271.232171
